In [1]:
import torch
import torch.utils.data as Data
import torch.nn as nn
import torch.optim as optim

from collections import OrderedDict

和3.2一样，再次生成数据

In [2]:
num_inputs = 2
num_examples = 1000

true_w = [2, -3.4]
true_b = 4.2

features = torch.randn(num_examples, num_inputs, dtype=torch.float32)

labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.normal(0, 0.01, size=labels.size(), dtype=torch.float32)

读取数据，使用torch.utils.data，这里引入为Data

DataLoader是Pytorch数据集操作的核心，主要作用有（来自官方文档）：

- map-style and iterable-style datasets,

- customizing data loading order,

- automatic batching,

- single- and multi-process data loading,

- automatic memory pinning.

其构造函数如下：

DataLoader(dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=None,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None)
           
而TensorDataset的含义也非常直白，即由tensor组成的数据集，

In [3]:
batch_size = 10

# 特征和标签组合生成数据集
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量数据
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

与上一节构造的generator相同，DataLoader的本质也是一个Python iterable可迭代对象

In [4]:
for X, y in data_iter:
    print(X, y)
    break

tensor([[ 0.9247, -0.5052],
        [-0.1997, -1.1511],
        [-1.0551,  0.3544],
        [-2.1021, -0.5460],
        [-0.3002, -0.2954],
        [ 1.0257,  1.1582],
        [ 0.3600,  0.4307],
        [-0.7719, -0.0346],
        [ 1.4988,  1.0616],
        [-1.1047,  0.1846]]) tensor([7.7641, 7.7097, 0.8719, 1.8304, 4.6195, 2.3026, 3.4691, 2.7743, 3.5920,
        1.3557])


定义模型  

线性回归也可以表示为单层的神经网络，因此，以下将采用多种方法构建我们的模型:

- 继承nn.Module的方法

nn.Module是torch.nn的核心，一个抽象类，既可以表示神经网络中的某个层（layer），也可以表示一个包含很多层的神经网络。子类应该定义一些层以及返回输出的前向传播（forward）方法。

nn.Linear构造方法：  
CLASS torch.nn.Linear(in_features: int, out_features: int, bias: bool = True)

In [5]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()  # 继承超类
        self.linear = nn.Linear(n_feature, 1)  # 加入线性层
        
    # 前向传播函数，以x为参数编写
    # 问题：如果有很多层的话，这里的前向传播函数是不是就对应有多行计算呢？我觉得是的
    def forward(self, x):
        return self.linear(x)

In [6]:
net = LinearNet(num_inputs)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


- 使用nn.Sequential搭建网络的方法（常用）

Sequential是一个有序的容器，网络层将按照在传入Sequential的顺序依次被添加到计算图中。

In [7]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )

# 写法二
net = nn.Sequential()  # 各层后续使用add_module添加
net.add_module('linear', nn.Linear(num_inputs, 1))  # linear是该层的名字，add_module既可以添加层，也可以根据名字修改层
# net.add_module ......

# 写法三（支持使用OrderedDict构建层） OrderedDict: 键值对按插入顺序排列
net = nn.Sequential(OrderedDict([
    ('linear', nn.Linear(num_inputs, 1))
    # ...
]))

In [8]:
print(net)
print(net[0]) # 通过索引可以访问各层

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


使用net.parameters查看模型所有的可学习参数，此函数将返回一个生成器。

In [9]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 0.5360, -0.5033]], requires_grad=True)
Parameter containing:
tensor([0.0879], requires_grad=True)


PyTorch在init模块（torch.nn.init）中提供了多种参数初始化方法。

In [10]:
nn.init.normal_(net[0].weight, mean=0, std=0.01)  # 可见weight和bias是Linear()层的成员变量，不同层对成员变量命名不尽相同
nn.init.constant_(net[0].bias, val=0)

Parameter containing:
tensor([0.], requires_grad=True)

定义损失函数

In [11]:
loss = nn.MSELoss()

定义优化算法

In [12]:
optimizer = optim.SGD(net.parameters(), lr=0.03)  # 几个参数
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


训练模型

In [13]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))  # -1表示此处值通过另一维度推算出来
        optimizer.zero_grad()  # 这一步也可以置于step后，对于第一个epoch并无影响
        l.backward()  # 对模型参数求梯度
        optimizer.step()  # 优化器工作：更新模型参数值
    print("epoch %d, loss: %f" % (epoch, l.item()))

epoch 1, loss: 0.000344
epoch 2, loss: 0.000074
epoch 3, loss: 0.000075


查看最终训练结果

In [14]:
print(true_w, net[0].weight)
print(true_b, net[0].bias)

[2, -3.4] Parameter containing:
tensor([[ 2.0001, -3.3994]], requires_grad=True)
4.2 Parameter containing:
tensor([4.1989], requires_grad=True)
